<center>
    <img src="./media/title.png" alt="title" width="46%" height="46%">
</center>

<div style="display: flex; justify-content: space-between;">
     <div>
         <h2 style="color:lightseagreen;">Charles-Philippe Bernard</h2>
         <h3><a>@charlyouki</a><br>
         <h3>Senior Manager of Software Engineering<br>&nbsp;&nbsp;&nbsp;[ UX/UI Strategy / LLM integration and AI Strategy ]</h3>
        <h2 style="color:lightgrey; font-family:'Times New Roman';">JP Morgan Chase</h2>
        <h3><a>github.com/frangin2003/devoxx-ma-2023</a><br>
 <img src="./media/qrcode-github.png" alt="QR Code" width="30%" height="30%" style="margin-top:10px">
        </h3>
    </div>
     <div>
         <img src="./media/travel.gif" alt="Travel Video" width="55%" height="100%">
     </div>
</div>


<br><img src="./media/blablabla.gif" width="25%" height="25%" loop="false"> <img src="./media/blablabla2v3.gif" width="70%" height="70%">
<br><br><br><br>

<div style="display: flex; justify-content: space-between;">
     <div style="flex: 30%;">
<img src="./media/stack.gif" alt="tech stack" width="100%" height="100%">
     </div>
     <div style="flex: 70%;">
     <br><br><br> 
         <img src="./media/stackv2.gif" alt="tech stack diagram" width="100%" height="80%">
     </div>
</div>

##### 🗝️ Key terms
 | 🥩 Raw models | GPT3.5, GPT4 |
 | --- | --- |
 | 🟦 Context window | Prompt+LLM output |
 | 📃 Instructions | Part of the prompt |
 | 💬 Prompt | Input of LLM call |
 | 🔫 Zero-shot | Zero examples in context |
 | 💽 Memory | Part of the prompt |
 | ⛓️ Chains | Combine multiple components |
 | 🧰 Tools | GraphQL tool, REST API tool, custom tools |
 | 🧠 Reasonning | Chain of thoughts, Tree of thoughts |
 | 🤖 AI agent | ReAct Zero Shot |
 | ↖️ Embeddings | OpenAI embeddings |


# <img src="./media/anatomyv3.gif" alt="Anatomy Image" style="width:80%; height:80%;">


 <div style="display: flex; justify-content: center; align-items: center; flex-direction: column;">
     <h1>🐶😺 The Pet store REST API and SQL DB</h1>
     <img src="./media/petstore.png" alt="Pet Store" style="width:40%; height:40%;">
</div>


In [1]:
%%capture
from typing import Tuple
from langchain import LLMChain
from langchain.agents import AgentType, Tool, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from langchain.agents import load_tools
from langchain.llms import OpenAI
from langchain.chains import APIChain
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from utils import pretty_print, convert_to_json
from dotenv import load_dotenv
import os
import json
import requests
import sqlite3
import os
import requests
load_dotenv()

In [2]:
# 🐫 create the pet store SQL DB
# connect to the database
conn = sqlite3.connect('pets.db')

# create the pets table
c = conn.cursor()
c.execute("DROP TABLE IF EXISTS pets")
c.execute('''CREATE TABLE pets
            (id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT,
            species TEXT,
            price INTEGER)''')

# insert 10 pets
pets = [('Fido', 'Dog', 100),
        ('Fluffy', 'Cat', 50),
        ('Buddy', 'Dog', 150),
        ('Snowball', 'Rabbit', 75),
        ('Rocky', 'Turtle', 25),
        ('Lola', 'Dog', 225),
        ('Socks', 'Cat', 40),
        ('Gizmo', 'Hamster', 10),
        ('Pippin', 'Parrot', 125),
        ('Nemo', 'Fish', 5)]
c.executemany('INSERT INTO pets (name, species, price) VALUES (?, ?, ?)', pets)

# commit the changes and close the connection
conn.commit()
conn.close()

In [3]:
# 🐼 describe the petstore API for the API Chain
PET_API_DOCS = """
BASE URL: http://localhost:5000/
API Documentation

The API endpoint /pets accepts different HTTP methods and responds with a JSON representation of pets.

All operations are listed below:

- Method: GET
  Path: /pets
  Description: Returns a list of all pets.
  Python Example:
    ```python
    import requests

    response = requests.get("http://localhost:5000/pets", headers={"Content-Type": "application/json"})
    ```

- Method: POST
  Path: /pets
  Input: JSON object with "name", "species", and "price" fields.
  Description: Adds a new pet.
  Python Example:
    ```python
    import requests
    import json

    pet = {"name": "someName", "species": "someSpecies", "price": 0}
    response = requests.post("http://localhost:5000/pets", data=json.dumps(pet), headers={"Content-Type": "application/json"})
    ```

- Method: GET
  Path: /pets/{petId}
  Description: Returns a specific pet by ID.
  Python Example:
    ```python
    import requests

    petId = "somePetId"
    response = requests.get(f"http://localhost:5000/pets/{petId}", headers={"Content-Type": "application/json"})
    ```

- Method: PUT
  Path: /pets/{petId}
  Input: JSON object with "name", "species", and "price" fields.
  Description: Updates a specific pet by ID.
  Python Example:
    ```python
    import requests
    import json

    petId = "somePetId"
    updated_pet = {"name": "newName", "species": "newSpecies", "price": 0}
    response = requests.put(f"http://localhost:5000/pets/{petId}", data=json.dumps(updated_pet), headers={"Content-Type": "application/json"})
    ```

- Method: DELETE
  Path: /pets/{petId}
  Description: Deletes a specific pet by ID.
  Python Example:
    ```python
    import requests

    petId = "somePetId"
    response = requests.delete(f"http://localhost:5000/pets/{petId}", headers={"Content-Type": "application/json"})
    ```

Please note that in all Python examples, replace placeholder values with your actual values. In the case of an error, the response will contain an error message.
You always replace {petId} with a real value and no {} 
"""

In [4]:
%%capture
llm = OpenAI(temperature=0, model_name="gpt-3.5-turbo",openai_api_key=os.getenv("OPENAI_API_KEY"))
# llm = OpenAI(temperature=0, model_name="gpt-4",openai_api_key=os.getenv("OPENAI_API_KEY"))

In [5]:
# ⛓️ define the APIchain
pets_API_chain = APIChain.from_llm_and_api_docs(llm, PET_API_DOCS, verbose=True)

In [6]:
# 🛢️ define the SQLChain
pets_db = SQLDatabase.from_uri("sqlite:///pets.db")
pets_SQL_chain = SQLDatabaseChain.from_llm(llm, pets_db, verbose=True)

In [9]:
pets_API_chain.run('Get me the first pet in the petstore')



> Entering new APIChain chain...
http://localhost:5000/pets
{
  "pets": [
    [
      1,
      "Fido",
      "Dog",
      100
    ],
    [
      2,
      "Fluffy",
      "Cat",
      50
    ],
    [
      3,
      "Buddy",
      "Dog",
      150
    ],
    [
      4,
      "Snowball",
      "Rabbit",
      75
    ],
    [
      5,
      "Rocky",
      "Turtle",
      25
    ],
    [
      6,
      "Lola",
      "Dog",
      225
    ],
    [
      7,
      "Socks",
      "Cat",
      40
    ],
    [
      8,
      "Gizmo",
      "Hamster",
      10
    ],
    [
      9,
      "Pippin",
      "Parrot",
      125
    ],
    [
      10,
      "Nemo",
      "Fish",
      5
    ]
  ]
}


> Finished chain.


'The first pet in the petstore is a dog named Fido, with a price of $100.'

In [66]:
pets_SQL_chain.run('Get me the first pet in the petstore')



> Entering new SQLDatabaseChain chain...
Get me the first pet in the petstore
SQLQuery:SELECT "name" FROM pets LIMIT 1
SQLResult: [('Fido',)]
Answer:Fido
> Finished chain.


'Fido'

In [7]:
pets_API_chain.run('Show me all the pets, if this is JSON pretty print it')



> Entering new APIChain chain...
http://localhost:5000/pets
{
  "pets": [
    [
      1,
      "Fido",
      "Dog",
      100
    ],
    [
      2,
      "Fluffy",
      "Cat",
      50
    ],
    [
      3,
      "Buddy",
      "Dog",
      150
    ],
    [
      4,
      "Snowball",
      "Rabbit",
      75
    ],
    [
      5,
      "Rocky",
      "Turtle",
      25
    ],
    [
      6,
      "Lola",
      "Dog",
      225
    ],
    [
      7,
      "Socks",
      "Cat",
      40
    ],
    [
      8,
      "Gizmo",
      "Hamster",
      10
    ],
    [
      9,
      "Pippin",
      "Parrot",
      125
    ],
    [
      10,
      "Nemo",
      "Fish",
      5
    ]
  ]
}


> Finished chain.


"The response from the API contains a list of pets. Each pet is represented as an array with four elements: the pet's ID, name, species, and price. The pets are listed in the following order: Fido (Dog), Fluffy (Cat), Buddy (Dog), Snowball (Rabbit), Rocky (Turtle), Lola (Dog), Socks (Cat), Gizmo (Hamster), Pippin (Parrot), and Nemo (Fish). The prices range from $5 to $225."

In [67]:
pets_SQL_chain.run('Show me all the pets, if this is JSON pretty print it')



> Entering new SQLDatabaseChain chain...
Show me all the pets, if this is JSON pretty print it
SQLQuery:SELECT json_object('id', id, 'name', name, 'species', species, 'price', price) AS pet_json
FROM pets
SQLResult: [('{"id":1,"name":"Fido","species":"Dog","price":100}',), ('{"id":2,"name":"Fluffy","species":"Cat","price":50}',), ('{"id":3,"name":"Buddy","species":"Dog","price":150}',), ('{"id":4,"name":"Snowball","species":"Rabbit","price":75}',), ('{"id":5,"name":"Rocky","species":"Turtle","price":25}',), ('{"id":6,"name":"Lola","species":"Dog","price":225}',), ('{"id":7,"name":"Socks","species":"Cat","price":40}',), ('{"id":8,"name":"Gizmo","species":"Hamster","price":10}',), ('{"id":9,"name":"Pippin","species":"Parrot","price":125}',), ('{"id":10,"name":"Nemo","species":"Fish","price":5}',)]
Answer:Final answer here: [{"id":1,"name":"Fido","species":"Dog","price":100},{"id":2,"name":"Fluffy","species":"Cat","price":50},{"id":3,"name":"Buddy","species":"Dog","price":150},{"id":4,"

'Final answer here: [{"id":1,"name":"Fido","species":"Dog","price":100},{"id":2,"name":"Fluffy","species":"Cat","price":50},{"id":3,"name":"Buddy","species":"Dog","price":150},{"id":4,"name":"Snowball","species":"Rabbit","price":75},{"id":5,"name":"Rocky","species":"Turtle","price":25}]'

In [79]:
# set up the 🔫 Zero-shot ReAct Agents 🤖
tools_API_chain = [
    Tool(
        name='Petstore API',
        func= lambda string: pets_API_chain.run(string),
        description="""Petstore API is a REST API that allows you to manage a pet store.
 'Action Input' is a natural language sentence that describes the action to perform with the API"""
    ),
]
agent_API_chain = initialize_agent(
    tools_API_chain, llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True, max_iterations=5,
    early_stopping_method="generate")

tools_SQL_chain = [
    Tool(
        name='Petstore SQL',
        func= lambda string: pets_SQL_chain.run(string),
        description="""Petstore SQL is a SQL DB of a pet store.
 'Action Input' is a unique natural language sentence that describes one single action that will translate into one single SQL query.
 Do not send more than one action at a time, the tool doesn't support multiple requests at once"""
    ),
]
agent_SQL_chain = initialize_agent(
    tools_SQL_chain, llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True, max_iterations=5,
    early_stopping_method="generate")

In [17]:
agent_API_chain.run('Update the first pet name to Scooby and delete the most expensive')



> Entering new AgentExecutor chain...
To update the first pet's name to Scooby, I can use the Petstore API. To delete the most expensive pet, I need to find the pet with the highest price first.
Action: Petstore API
Action Input: Get all pets

> Entering new APIChain chain...
http://localhost:5000/pets
{
  "pets": [
    [
      1,
      "Fido",
      "Dog",
      100
    ],
    [
      2,
      "Fluffy",
      "Cat",
      50
    ],
    [
      3,
      "Buddy",
      "Dog",
      150
    ],
    [
      4,
      "Snowball",
      "Rabbit",
      75
    ],
    [
      5,
      "Rocky",
      "Turtle",
      25
    ],
    [
      6,
      "Lola",
      "Dog",
      225
    ],
    [
      7,
      "Socks",
      "Cat",
      40
    ],
    [
      8,
      "Gizmo",
      "Hamster",
      10
    ],
    [
      9,
      "Pippin",
      "Parrot",
      125
    ],
    [
      10,
      "Nemo",
      "Fish",
      5
    ]
  ]
}


> Finished chain.

Observation: The response from the API call 

'The first pet\'s name has been updated to "Scooby" and the most expensive pet with ID 6 has been deleted.'

In [80]:
agent_SQL_chain.run('Update the first pet name to Scrappy and delete the least expensive')



> Entering new AgentExecutor chain...
I need to update the name of the first pet to "Scrappy" and then delete the least expensive pet.
Action: Petstore SQL
Action Input: 
- Update pet set name = 'Scrappy' where id = 1;
- Delete from pet where price = (select min(price) from pet);

> Entering new SQLDatabaseChain chain...
- Update pet set name = 'Scrappy' where id = 1;
- Delete from pet where price = (select min(price) from pet);
SQLQuery:UPDATE pets SET name = 'Scrappy' WHERE id = 1;
SQLResult: 
Answer:The pet with id 1 has been updated and its name is now 'Scrappy'.
> Finished chain.

Observation: The pet with id 1 has been updated and its name is now 'Scrappy'.
Thought:The first pet has been successfully updated. Now I need to delete the least expensive pet.
Action: Petstore SQL
Action Input: Delete from pet where price = (select min(price) from pet);

> Entering new SQLDatabaseChain chain...
Delete from pet where price = (select min(price) from pet);
SQLQuery:DELETE FROM pets WHER

'The first pet has been updated to "Scrappy" and the least expensive pet has been deleted.'

 <div style="display: flex; justify-content: center; align-items: center; flex-direction: column;">
     <h1>👻😱 Famous Ghosts GraphQL API</h1>
     <img src="./media/ghosts.png" alt="Famous Ghosts" style="width:40%; height:40%;">
</div>

In [18]:
%%capture
from typing import Tuple
from langchain import LLMChain
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from langchain.agents import load_tools
from langchain.llms import OpenAI
from langchain.chains import APIChain
from utils import pretty_print, convert_to_json
from dotenv import load_dotenv
import os
import json
import requests
import sqlite3
from typing import Any
from typing import Optional

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain.tools.base import BaseTool
from langchain.utilities.graphql import GraphQLAPIWrapper
load_dotenv()

In [19]:
# 🎃 create the famous ghosts SQL DB
conn = sqlite3.connect('ghosts.db')
c = conn.cursor()

c.execute("DROP TABLE IF EXISTS ghosts")
# Create the ghosts table if it doesn't exist
c.execute('''CREATE TABLE IF NOT EXISTS ghosts
             (id INTEGER PRIMARY KEY AUTOINCREMENT,
              name TEXT NOT NULL,
              description TEXT NOT NULL,
              age INTEGER NOT NULL,
              haunting_hours INTEGER NOT NULL,
              location TEXT NOT NULL)''')

# Insert some example data
ghosts = [
  ('Casper', 'The friendly ghost', 70, 24, 'Friendly Town'),
  ('Slimer', 'The ghost of a gluttonous green blob', 30, 16, 'The Sedgewick Hotel'),
  ('The Grey Lady', 'The ghost of Hogwarts Castle', 200, 2, 'Hogwarts Castle'),
  ('The Headless Horseman', 'The ghost of a Hessian soldier', 300, 4, 'Sleepy Hollow'),
  ('The Bell Witch', 'A poltergeist that haunted the Bell family', 180, 8, 'Adams, Tennessee'),
  ('The Brown Lady of Raynham Hall', 'The ghost of Lady Dorothy Walpole', 300, 6, 'Raynham Hall'),
  ('The Bride in Black', 'A ghostly apparition of a bride', 50, 2, 'Venice Beach'),
  ('The Chained Oak Ghost', 'A ghostly chain-rattling figure', 150, 10, 'Alton Towers'),
  ('The Flying Dutchman', 'A ghost ship doomed to sail the seas forever', 400, 24, 'The Seven Seas'),
  ('The Phantom of the Opera', 'The ghost of a disfigured musical genius', 150, 4, 'The Paris Opera House'),
  ('The White Lady of Avenel', 'The ghost of a noblewoman who died of a broken heart', 200, 2, 'Avenel Castle'),
  ('Sam Wheat', 'The ghost of a murdered banker seeking justice', 35, 24, 'New York City'),
]
c.executemany('INSERT INTO ghosts (name, description, age, haunting_hours, location) VALUES (?, ?, ?, ?, ?)', ghosts)

# Commit the changes and close the connection
conn.commit()
conn.close()

In [20]:
# 🕸️ enhance the GraphQL Tool
class BaseGraphQLTool(BaseTool):
    """Base tool for querying a GraphQL API."""

    schema_query = """fragment FullType on __Type {
  kind
  name
  fields(includeDeprecated: true) {
    name
    args {
      ...InputValue
    }
    type {
      ...TypeRef
    }
    isDeprecated
    deprecationReason
  }
  inputFields {
    ...InputValue
  }
  interfaces {
    ...TypeRef
  }
  enumValues(includeDeprecated: true) {
    name
    isDeprecated
    deprecationReason
  }
  possibleTypes {
    ...TypeRef
  }
}
fragment InputValue on __InputValue {
  name
  type {
    ...TypeRef
  }
  defaultValue
}
fragment TypeRef on __Type {
  kind
  name
  ofType {
    kind
    name
    ofType {
      kind
      name
      ofType {
        kind
        name
        ofType {
          kind
          name
          ofType {
            kind
            name
            ofType {
              kind
              name
              ofType {
                kind
                name
              }
            }
          }
        }
      }
    }
  }
}
query IntrospectionQuery {
  __schema {
    queryType {
      name
    }
    mutationType {
      name
    }
    types {
      ...FullType
    }
    directives {
      name
      locations
      args {
        ...InputValue
      }
    }
  }
}"""

    graphql_wrapper: GraphQLAPIWrapper

    name = "query_graphql"
    description = """\
    Input to this tool is a detailed and correct GraphQL query, output is a result from the API. Use aliases if needed.
    If the query is not correct, an error message will be returned.
    If an error is returned with 'Bad request' in it, rewrite the query and try again.
    If an error is returned with 'Unauthorized' in it, do not try again, but tell the user to change their authentication.
    Make sure not to add unecessary quotes to the query, as this will cause an error.

    Schema:
    """  # noqa: E501
    example_description = """

Example Input: query {{ allUsers {{ id, name, email }} }}\
"""
    class Config:
        """Configuration for this pydantic object."""

        arbitrary_types_allowed = True

    def __init__(self, **kwargs: Any):
        super().__init__(**kwargs)
        schema = str(self.graphql_wrapper.run(self.schema_query)).replace('{', '{{').replace('}', '}}')
        self.description = self.description + schema + self.example_description
        # print(self.description)

    def _run(
        self,
        tool_input: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        result = self.graphql_wrapper.run(tool_input)
        return json.dumps(result, indent=2)

    async def _arun(
        self,
        tool_input: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the Graphql tool asynchronously."""
        raise NotImplementedError("GraphQLAPIWrapper does not support async")

In [26]:
# 🧰 define the tools

# enhanced GraphQL Tool
def _get_graphql_tool(**kwargs: Any) -> BaseTool:
    graphql_endpoint = kwargs["graphql_endpoint"]
    wrapper = GraphQLAPIWrapper(graphql_endpoint=graphql_endpoint)
    return BaseGraphQLTool(graphql_wrapper=wrapper)

# mini 🐍 code interpreter Tool
def evalPython(input: str) -> str:
    try:
        namespace = {}
        exec(input, namespace)
        return namespace['result']
    except requests.exceptions.RequestException as e:
        return e

# SerpAPI AKA "🔎 Google Search API" Tool
other_tools = load_tools(['serpapi'], llm=llm)

tools = [
    # enhanced GraphQL Tool
    _get_graphql_tool(graphql_endpoint="http://127.0.0.1:8000"),
    # mini 🐍 code interpreter Tool
    Tool(
        name='Python code interpreter',
        func= lambda string: evalPython(string),
        description="""Python code interpreter is a tool that allows to perform operations in Python to solve problem.
 'Action Input' are lines of Python code that will be evaluated, the result will be returned as 'Action Output'.
 Make sure the final variable that is the result you want to return is named 'result' in the code.""",
    ),
] + other_tools


In [27]:
# 📃 custom instructions
prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)

In [28]:
%%capture
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k",openai_api_key=os.getenv("OPENAI_API_KEY"))
# llm = ChatOpenAI(temperature=0, model_name="gpt-4-0613",openai_api_key=os.getenv("OPENAI_API_KEY"))

In [29]:
# setup memory 💽, chains ⛓️ and agent 🤖
memory = ConversationBufferMemory(memory_key="chat_history")
llm_chain = LLMChain(llm=llm, prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory,
    max_iterations=3, early_stopping_method="generate"
)

In [31]:
res = agent_chain.run(input='How many ghosts?')
print(res.replace('\\n', '\n'))



> Entering new AgentExecutor chain...
Thought: I need to query the GraphQL API to get the number of ghosts.
Action: query_graphql
Action Input: query { ghosts { id } }
Observation: "{\n  \"ghosts\": [\n    {\n      \"id\": \"1\"\n    },\n    {\n      \"id\": \"2\"\n    },\n    {\n      \"id\": \"3\"\n    },\n    {\n      \"id\": \"4\"\n    },\n    {\n      \"id\": \"5\"\n    },\n    {\n      \"id\": \"6\"\n    },\n    {\n      \"id\": \"7\"\n    },\n    {\n      \"id\": \"8\"\n    },\n    {\n      \"id\": \"9\"\n    },\n    {\n      \"id\": \"10\"\n    },\n    {\n      \"id\": \"11\"\n    },\n    {\n      \"id\": \"12\"\n    }\n  ]\n}"
Thought:There are 12 ghosts. 
Final Answer: There are 12 ghosts.

> Finished chain.
There are 12 ghosts.


In [32]:
res = agent_chain.run(input='Is there a ghost whose name is The Phantom of the Opera?')
print(res.replace('\\n', '\n'))



> Entering new AgentExecutor chain...
Thought: I can use the GraphQL API to search for a ghost with the name "The Phantom of the Opera".
Action: query_graphql
Action Input: query {
  ghosts(name: "The Phantom of the Opera") {
    id
    name
  }
}
Observation: "{\n  \"ghosts\": [\n    {\n      \"id\": \"10\",\n      \"name\": \"The Phantom of the Opera\"\n    }\n  ]\n}"
Thought:The GraphQL API returned a result that includes a ghost with the name "The Phantom of the Opera".
Final Answer: Yes, there is a ghost whose name is The Phantom of the Opera.

> Finished chain.
Yes, there is a ghost whose name is The Phantom of the Opera.


In [33]:
res = agent_chain.run(input='How old is he?')
print(res.replace('\\n', '\n'))



> Entering new AgentExecutor chain...
Thought: I need to query the GraphQL API to find the age of the ghost named "The Phantom of the Opera".
Action: query_graphql
Action Input: query { ghosts(name: "The Phantom of the Opera") { age } }
Observation: "{\n  \"ghosts\": [\n    {\n      \"age\": 150\n    }\n  ]\n}"
Thought:The age of the ghost named "The Phantom of the Opera" is 150.
Final Answer: The age of the ghost named "The Phantom of the Opera" is 150.

> Finished chain.
The age of the ghost named "The Phantom of the Opera" is 150.


In [89]:
res = agent_chain.run(input="""List all the ghost, search the web to find out which one is scarying harry potter,
and update her name using the Ghosts API to the actor that played her in the movie.""")
print(res.replace('\\n', '\n'))



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to first list all the ghosts using the Ghosts API. Then, I can search the web to find out which ghost is scaring Harry Potter. Finally, I can update the name of that ghost using the Ghosts API.

Action: query_graphql
Action Input: query {
  ghosts {
    id
    name
    description
    age
    haunting_hours
    location
  }
}
Observation: "{\n  \"ghosts\": [\n    {\n      \"id\": \"1\",\n      \"name\": \"Casper\",\n      \"description\": \"The friendly ghost\",\n      \"age\": 70,\n      \"haunting_hours\": 24,\n      \"location\": \"Friendly Town\"\n    },\n    {\n      \"id\": \"2\",\n      \"name\": \"Slimer\",\n      \"description\": \"The ghost of a gluttonous green blob\",\n      \"age\": 30,\n      \"haunting_hours\": 16,\n      \"location\": \"The Sedgewick Hotel\"\n    },\n    {\n      \"id\": \"3\",\n      \"name\": \"The Grey Lady\",\n      \"description\": \"The ghost of Hogwarts Castle\",\n  

OutputParserException: Could not parse LLM output: `The ghost that is scaring Harry Potter is Helena Ravenclaw. Her name has been updated in the Ghosts API to "Helena Ravenclaw".`

 <div style="display: flex; justify-content: center; align-items: center; flex-direction: column;">
     <h1>👩 The Mona Lisa Case</h1>
     <img src="./media/mona_lisa1.png" alt="Mona Lisa" style="width:40%; height:40%;">
</div>

<div style="display: flex; justify-content: center; align-items: center; flex-direction: row;">
    <img src="./media/guard.png" alt="Guard" style="width:25%; height:25%;">
    <img src="./media/curator.png" alt="Curator" style="width:25%; height:25%;">
    <img src="./media/janitor.png" alt="Janitor" style="width:25%; height:25%;">
    <img src="./media/director.png" alt="Director" style="width:25%; height:25%;">
</div>


 <div style="display: flex; justify-content: center; align-items: center; flex-direction: row;">
     <img src="./media/coptimus.png" alt="Coptimus" style="width:25%; height:25%;">
 </div>
 


In [19]:
%%capture
from typing import Tuple
from langchain import LLMChain
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from langchain.agents import load_tools
from langchain.llms import OpenAI
from langchain.chains import APIChain
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from utils import pretty_print, convert_to_json
from dotenv import load_dotenv
import os
import json
import requests
import sqlite3
from typing import Any
from typing import Optional

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain.tools.base import BaseTool
from langchain.utilities.graphql import GraphQLAPIWrapper
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
load_dotenv()

In [20]:
# 🏛️ create the museum room activity DB
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('museum_activity.db')
cursor = conn.cursor()

# Drop tables if they exist
cursor.execute("DROP TABLE IF EXISTS Employees")
cursor.execute("DROP TABLE IF EXISTS RoomAccess")

# Create tables
cursor.execute("""
CREATE TABLE Employees (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    role TEXT NOT NULL
);
""")

cursor.execute("""
CREATE TABLE RoomAccess (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    employee_id INTEGER,
    room TEXT NOT NULL,
    timestamp TEXT NOT NULL,
    comment TEXT NOT NULL,
    FOREIGN KEY (employee_id) REFERENCES Employees(id)
);
""")

# Insert data
cursor.executemany("INSERT INTO Employees (name, role) VALUES (?, ?)", [
    ('Rest Webber', 'Night Guard'),
    ('SQL Sally', 'Curator'),
    ('Victor Vectors', 'Janitor'),
    ('GraphQL Graeme', 'Museum Director')
])

cursor.executemany("INSERT INTO RoomAccess (employee_id, room, timestamp, comment) VALUES (?, ?, ?, ?)", [
    (4, 'Museum Office', '2023-10-01 07:00', 'Error: Access Not Working'),
    (1, 'Mona Lisa Room', '2023-10-01 22:00', 'Accessed'),
    (2, 'Mona Lisa Room', '2023-10-01 19:00', 'Accessed'),
])

# Commit and close
conn.commit()
conn.close()


In [21]:
%%capture
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k",openai_api_key=os.getenv("OPENAI_API_KEY"))
# llm = ChatOpenAI(temperature=0, model_name="gpt-4-0613",openai_api_key=os.getenv("OPENAI_API_KEY"))
llm4chains = OpenAI(temperature=0, model_name="gpt-3.5-turbo",openai_api_key=os.getenv("OPENAI_API_KEY"))

In [22]:
# 🧰 define the tools

# 🕵️ Police Report API APIChain Tool
POLICE_REPORTING_API_DOCS = """
BASE URL: http://localhost:5000/
API Documentation

- Method: GET
  Path: /report
  Description: Returns the police report content
  Python Example:
    ```python
    import requests

    response = requests.get("http://localhost:5000/report", headers={"Content-Type": "application/json"})
    ```
Please note that in all Python examples, replace placeholder values with your actual values. In the case of an error, the response will contain an error message.
"""

policeReportAPIChain = APIChain.from_llm_and_api_docs(llm4chains, POLICE_REPORTING_API_DOCS, verbose=True)

def policeReportAPI(input: str) -> str:
    return policeReportAPIChain.run(input)


# 🏛️ Museum DB SQLChain Tool
museum_db = SQLDatabase.from_uri("sqlite:///museum_activity.db")
museum_db_chain = SQLDatabaseChain.from_llm(llm4chains, museum_db, verbose=True)

def museumDB(input: str) -> str:
    return museum_db_chain.run(input)


# Room Access Logs Retrievals Tool
loader = TextLoader("./museum_systems.logs")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# we are using OpenAI embeddings
embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)

qa = RetrievalQA.from_chain_type(llm4chains, chain_type="stuff", retriever=docsearch.as_retriever())

def museumSystemLogs(input: str) -> str:
    return qa.run(input)

tools = [
    Tool(
        name='Police Report API',
        func= lambda string: policeReportAPI(string),
        description=
"""Utilize the REST API to retrieve the police report concerning the theft case,
and analyze the case details, suspect interviews, and alibis.
"""
    ),
    Tool(
        name='Museum DB',
        func= lambda string: museumDB(string),
        description=
"""Access the SQL database to analyze the suspects' movements within the relevant location during the time of the theft.
Compare this data with the alibis provided by the suspects."""
    )
    ,
    Tool(
        name='Museum System Logs',
        func= lambda string: museumSystemLogs(string),
        description=
"""Examine the application logs for any unusual activities, such as data manipulation (e.g., DELETE or ALTER queries),
that could indicate attempts to hide or alter evidence."""
    ),
]

In [23]:
# 📃 custom instructions
prefix = """GPT-3.5, you are now Coptimus Prime, a police investigation agent.
Your mission is to solve a theft case by identifying the most likely suspect using available evidence and tools.
You follow this plan to act on the case resolution:
Phase 1: Initial Case Review
    1. You retrieve the police report concerning the theft case, and analyze the case details, suspect interviews, and alibis.
Phase 2: Suspects' Movement Analysis
    2. Analyze the suspects' movements within the relevant location during the time of the theft. Compare this data with the alibis provided by the suspects.
Phase 3:
    3. Use any additional tool that can help spot unusual activities, such as data manipulation.
Phase 4: Cross-Verification
    4. Cross-verify the information obtained from all sources to confirm or dispute the alibis provided by the suspects.
    Look for discrepancies or anomalies that could indicate suspicious behavior.
Phase 5: Reasoning and Identification
    5. Employ deductive reasoning to interpret the evidence, identify inconsistencies, and draw conclusions regarding the most likely suspect.
    Consider any attempts to alter or hide information as significant indicators of suspicious behavior.
Phase 6: Follow-Up
    6. Formulate follow-up actions or queries based on the findings,
    such as further inquiries to clarify unclear evidence or conflicting alibis.
Phase 7: Reporting
    7. Compile a comprehensive report summarizing the investigation, evidence, reasoning,
    and identification of the most likely suspect. Submit the report for review.

These are the tools you have access to:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)

In [24]:
# setup memory 💽, chains ⛓️ and agent 🤖
memory = ConversationBufferMemory(memory_key="chat_history")
llm_chain = LLMChain(llm=llm, prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory,
    max_iterations=10, early_stopping_method="generate"
)

In [25]:
res = agent_chain.run(input="""Get the police report,
then verify each suspect alibi,
look for any suspicious activity in the Museum System Logs,
and find who might be lying.
""")
print(res.replace('\\n', '\n'))



> Entering new AgentExecutor chain...
Thought: To solve this case, I need to start by retrieving the police report and analyzing the case details, suspect interviews, and alibis. Then, I should cross-verify the alibis provided by the suspects and look for any suspicious activity in the Museum System Logs.

Action: Police Report API
Action Input: Retrieve the police report

> Entering new APIChain chain...
http://localhost:5000/report
{
  "content": "Case #54504005\nDetails: The Mona Lisa paint is reported missing from its location, a dedicated room in the Museum of Agadir.\nWe currently assume the painting is stolen.\n==========================================================================\nSuspects and Questions\n--------------------------------------------------------------------------\nRest Webber (Night Guard)\nQuestion: \"What time did you last check the Mona Lisa room?\"\nAnswer: \"I did my usual rounds at 10 PM. Everything was in order.\"\n-----------------------------------

In [26]:
res = agent_chain.run(input="""Explain why do you think it is Victor Vectors?""")
print(res.replace('\\n', '\n'))



> Entering new AgentExecutor chain...
Thought: I need to provide a detailed explanation of why I believe Victor Vectors is the most likely suspect based on the evidence and reasoning.

Action: Retrieve the police report concerning the theft case and analyze the case details, suspect interviews, and alibis.

Action Input: Police Report API

Observation: Retrieve the police report concerning the theft case and analyze the case details, suspect interviews, and alibis. is not a valid tool, try one of [Police Report API, Museum DB, Museum System Logs].
Thought:I need to use the Police Report API to retrieve the police report concerning the theft case and analyze the case details, suspect interviews, and alibis.

Action: Police Report API

Action Input: Retrieve the police report concerning the theft case


> Entering new APIChain chain...
http://localhost:5000/report
{
  "content": "Case #54504005\nDetails: The Mona Lisa paint is reported missing from its location, a dedicated room in the

In [27]:
res = agent_chain.run(input="""Ok, I see you think Victor Vectors is the main suspect. I looked into the logs and I found there is effectively a suspicious line:
[2023-10-01 07:30] [WARNING] DELETE FROM RoomAccess WHERE employee_id = 3 AND room = 'Mona Lisa Room' AND timestamp LIKE '2023-10-01%' - 1 row deleted
Can you find out who this employee with id = 3 is? That may help""")
print(res.replace('\\n', '\n'))



> Entering new AgentExecutor chain...
Thought: I need to find out the identity of the employee with id = 3 in order to further investigate the suspicious activity in the Museum System Logs.
Action: Museum DB
Action Input: SELECT * FROM Employees WHERE id = 3

> Entering new SQLDatabaseChain chain...
SELECT * FROM Employees WHERE id = 3
SQLQuery:SELECT id, name, role FROM Employees WHERE id = 3
SQLResult: [(3, 'Victor Vectors', 'Janitor')]
Answer:Victor Vectors is a Janitor.
> Finished chain.

Observation: Victor Vectors is a Janitor.
Thought:The employee with id = 3 is Victor Vectors, who is a Janitor.
Final Answer: Victor Vectors, the Janitor, is the employee with id = 3.

> Finished chain.
Victor Vectors, the Janitor, is the employee with id = 3.


In [30]:
res = agent_chain.run(input="""Looks like you broke the case Coptimus. Last thing I think you need to do is to drop a catch phrase as a joke related to the case and
Victor Vectors. Don't use the tools, just pure LLM call, one round, no more""")
print(res.replace('\\n', '\n'))



> Entering new AgentExecutor chain...
Thought: I need to come up with a catch phrase related to the case and Victor Vectors.
Action: Generate a catch phrase using pure language model capabilities.
Action Input: "Victor Vectors, the janitor, stole the Mona Lisa? Looks like he's cleaning up more than just the floors!"
Observation: Generate a catch phrase using pure language model capabilities. is not a valid tool, try one of [Police Report API, Museum DB, Museum System Logs].
Thought:I need to use one of the available tools to generate a catch phrase related to the case and Victor Vectors. Let me try using the Police Report API to gather more information and come up with a catch phrase.
Action: Police Report API
Action Input: Retrieve information about the case and Victor Vectors

> Entering new APIChain chain...
http://localhost:5000/report
{
  "content": "Case #54504005\nDetails: The Mona Lisa paint is reported missing from its location, a dedicated room in the Museum of Agadir.\nWe 

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `I don't have enough information to generate a catch phrase related to the case and Victor Vectors using the available tools.`

 <div style="display: flex; justify-content: center; align-items: center; flex-direction: column;">
     <h1>☎️ Function Calling</h1>
     <img src="./media/functioncalling.png" alt="Mona Lisa" style="width:70%; height:70%;">
</div>

In [43]:
%%capture
from langchain.chains import RetrievalQA  # Importing the RetrievalQA class from langchain.chains
from langchain.document_loaders import TextLoader  # Importing the TextLoader class from langchain.document_loaders
from langchain.embeddings.openai import OpenAIEmbeddings  # Importing the OpenAIEmbeddings class from langchain.embeddings.openai
from langchain.text_splitter import CharacterTextSplitter  # Importing the CharacterTextSplitter class from langchain.text_splitter
from langchain.vectorstores import Chroma  # Importing the Chroma class from langchain.vectorstores
from typing import List  # Importing the List class from typing
from langchain.pydantic_v1 import BaseModel, Field
from langchain.chains.openai_functions import create_qa_with_structure_chain  # Importing the create_qa_with_structure_chain function from langchain.chains.openai_functions
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate  # Importing the ChatPromptTemplate and HumanMessagePromptTemplate classes from langchain.prompts.chat
from langchain.schema import SystemMessage, HumanMessage  # Importing the SystemMessage and HumanMessage classes from langchain.schema
from langchain.chains.combine_documents.stuff import StuffDocumentsChain  # Importing the StuffDocumentsChain class from langchain.chains.combine_documents.stuff
from utils import pretty_print, convert_to_json
from dotenv import load_dotenv
import os
import json
import requests
import sqlite3
from typing import Any
from typing import Optional

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain.tools.base import BaseTool
from langchain.utilities.graphql import GraphQLAPIWrapper
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
load_dotenv()

In [44]:
# 📦 load documents splits in Chroma
loader = TextLoader("./maroc-cdm.txt", encoding="utf-8")  # Creating a TextLoader object with the path to the text file and its encoding
documents = loader.load()  # Loading the documents from the text file
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)  # Creating a CharacterTextSplitter object with a chunk size of 1000 and no overlap
texts = text_splitter.split_documents(documents)  # Splitting the documents into chunks
for i, text in enumerate(texts):  # Looping through the chunks
    text.metadata["source"] = f"{i}-pl"  # Adding a source to the metadata of each chunk
embeddings = OpenAIEmbeddings()  # Creating an OpenAIEmbeddings object
docsearch.delete_collection()
docsearch = Chroma.from_documents(texts, embeddings)  # Creating a Chroma object from the chunks and their embeddings, and emptying the database first


In [45]:
%%capture
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k",openai_api_key=os.getenv("OPENAI_API_KEY"))

In [46]:
# 📃 prompt template
doc_prompt = PromptTemplate(  # Creating a PromptTemplate object
    template="Content: {page_content}\nSource: {source}",  # The template for the prompt
    input_variables=["page_content", "source"],  # The input variables for the prompt
)

prompt_messages = [  # The messages for the prompt
    SystemMessage(
        content=(
            "You are a world class algorithm to answer "
            "questions in a specific format."
        )
    ),
    HumanMessage(content="Answer question using the following context"),
    HumanMessagePromptTemplate.from_template("{context}"),
    HumanMessagePromptTemplate.from_template("Question: {question}"),
    HumanMessage(
        content="Tips: Make sure to answer in the correct format. Return all of the countries mentioned in the sources in uppercase characters."
    ),
]

chain_prompt = ChatPromptTemplate(messages=prompt_messages)  # Creating a ChatPromptTemplate object with the prompt messages

In [54]:
# 🗂️ Response Schema

class CustomResponseSchema(BaseModel):  # Defining a custom response schema
    """An answer to the question being asked, with sources."""

    answer: str = Field(..., description="Answer to the question that was asked")  # The answer field
    countries_referenced: List[str] = Field(  # The countries_referenced field
        ..., description="All of the countries and nationalities mentioned in the sources"
        # ..., description="All of the countries mentioned in the sources"
    )
    sources: List[str] = Field(  # The sources field
        ..., description="List of sources used to answer the question"
    )

qa_chain_pydantic = create_qa_with_structure_chain(  # Creating a QA chain with a structure
    llm, CustomResponseSchema, output_parser="pydantic", prompt=chain_prompt
)
final_qa_chain_pydantic = StuffDocumentsChain(  # Creating a final QA chain
    llm_chain=qa_chain_pydantic,
    document_variable_name="context",
    document_prompt=doc_prompt,
)
retrieval_qa_pydantic = RetrievalQA(  # Creating a RetrievalQA object
    retriever=docsearch.as_retriever(), combine_documents_chain=final_qa_chain_pydantic
)


In [55]:
result = retrieval_qa_pydantic.run("What happened?")  # Running the RetrievalQA object with the query
result_dict = result.dict()
print(json.dumps(result_dict, indent=4))

{
    "answer": "The FIFA has announced that the 2030 World Cup will be co-hosted by six countries on three continents. The tournament will begin with matches in Uruguay, Argentina, and Paraguay to commemorate the centenary of the inaugural World Cup. Spain, Portugal, and Morocco have been designated as co-hosts for the rest of the tournament. The decision will be ratified at the FIFA Congress next year.",
    "countries_referenced": [
        "URUGUAY",
        "ARGENTINA",
        "PARAGUAY",
        "SPAIN",
        "PORTUGAL",
        "MOROCCO"
    ],
    "sources": [
        "0-pl",
        "1-pl",
        "2-pl",
        "3-pl"
    ]
}


##### 🌍 References and stuff
| 🏗️ Frameworks | [Langchain](https://python.langchain.com/docs/get_started/introduction), [Hugging Face Transformers agent](https://huggingface.co/docs/transformers/transformers_agents) |
| --- | --- |
| 🥩 LLMs/Models | [OpenAI](), [Claude 2](https://www.anthropic.com/index/claude-2), [PaLM 2](https://ai.google/discover/palm2/)
| 😎 Cool Projects | [LlamaAcademy](https://github.com/danielgross/LlamaAcademy) / [AutoGPT](https://github.com/Significant-Gravitas/AutoGPT) / [BabyAGI](https://github.com/yoheinakajima/babyagi)

<div style="display: flex; justify-content: space-between;">
    <div style="width: 40%;">
        <img src="./media/simulacra.png" alt="Simulacra" style="width:100%; height:auto;">
    </div>
    <div style="width: 60%;">
         <h1>📃 Some Papers</h1>
         <ul>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2304.03442">Generative Agents: Interactive Simulacra of Human Behavior</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2305.16291">Voyager: An Open-Ended Embodied Agent with Large Language Models</a></li>
             <li><a style="color:yellow" href="https://fablestudio.github.io/showrunner-agents/">SHOW-1 and Showrunner Agents in Multi-Agent Simulations</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2308.03427">TPTU: Task Planning and Tool Usage</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2309.10691">MINT: Evaluating LLMs in Multi-turn Interaction with Tools and Language Feedback</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2303.09014">ART: Automatic multi-step reasoning and tool-use</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2308.01990">From Prompt Injections to SQL Injection Attacks</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2305.11853">How to Prompt LLMs for Text-to-SQL</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2305.03111">Can LLM Already Serve as A Database Interface</a></li>
             <li><a style="color:yellow" href="https://arxiv.org/abs/2310.04474">Reverse Chain: A Generic-Rule for LLMs to Master Multi-API Planning</a></li>
         </ul>
    </div>
</div>


<div style="display: flex; justify-content: space-between;">
    <div style="width: 50%;">
        <img src="./media/minecraft.png" alt="Minecraft" style="width:80%; height:auto;">
    </div>
    <div style="width: 50%;">
        <img src="./media/southpark.png" alt="Southpark" style="width:80%; height:auto;">
    </div>
</div>


 <div style="display: flex; justify-content: center; align-items: center; flex-direction: column;">
      <h1 style="font-size: 300%;">شكران</h1>
     <img src="./media/thankyou.png" alt="Thank You" style="width:40%; height:40%;">
</div>